In [22]:
# --- JUPYTER NOTEBOOK CELL ---
# Pruning algorithm with IMPROVED GRID CONTROL (distribution-free SAFE version)
# Requirements:
#   - A DataFrame `subset` from the previous cell with columns: ["height_m", "damage_per_unit", "C_n"].
#   - Optionally, subset.attrs["N_UNITS"] (else set below).
#
# Grid control:
#   GRID_MODE in {"none", "uniform_step", "target_edges", "scale_factor"}
# After densifying the grid, we recompute step margins and run distribution-free pruning
# using ONLY upward margins m_plus (safe). m_minus is diagnostic only.

import numpy as np
import pandas as pd

# =========================
# 0) USER SETTINGS
# =========================
# Number of time units n:
N_UNITS = 1  # <-- change if not set upstream

# Grid refinement mode:
GRID_MODE = "target_edges"               # "none" | "uniform_step" | "target_edges" | "scale_factor"

# Parameters for modes:
UNIFORM_STEP_M   = 0.10          # used only if GRID_MODE == "uniform_step" (e.g., 0.10 m)
TARGET_NUM_EDGES = 80            # used only if GRID_MODE == "target_edges" (points = edges+1)
EXTRA_PER_EDGE   = 2             # used only if GRID_MODE == "scale_factor" (new pts per original edge)

INCLUDE_ORIGINAL = False         # include original grid points in densified representation
ENFORCE_MONOTONE = True          # enforce nondecreasing interpolants for C_n and d(h) after interpolation

# Output formatting:
DEC_H = 3     # decimals for height
DEC_VAL = 3   # decimals for values

# =========================
# 1) SANITY CHECKS
# =========================
if "subset" not in globals():
    raise RuntimeError("Expected `subset` from the previous cell. Please run the loading cell first.")

base = subset.sort_values("height_m").reset_index(drop=True).copy()
required = {"height_m", "damage_per_unit", "C_n"}
missing = required - set(base.columns)
if missing:
    raise ValueError(f"`subset` is missing required columns: {missing}")
if base.shape[0] < 2 and GRID_MODE != "target_edges":
    raise ValueError("Need at least 2 base heights to densify (except for 'target_edges' mode).")

h_min, h_max = float(base["height_m"].min()), float(base["height_m"].max())
if not np.isfinite(h_min) or not np.isfinite(h_max) or h_min >= h_max:
    raise ValueError("Invalid or degenerate height range in `subset`.")

# =========================
# 2) BUILD NEW HEIGHT GRID
# =========================
def make_dense_heights(base_heights: np.ndarray) -> np.ndarray:
    base_heights = np.asarray(base_heights, dtype=float)
    if GRID_MODE == "none":
        new_h = base_heights.copy()

    elif GRID_MODE == "uniform_step":
        step = float(UNIFORM_STEP_M)
        if step <= 0:
            raise ValueError("UNIFORM_STEP_M must be > 0 for 'uniform_step' mode.")
        start, end = base_heights.min(), base_heights.max()
        num_pts = int(np.floor((end - start) / step)) + 1
        new_h = start + np.arange(num_pts) * step
        if new_h[-1] < end - 1e-12:
            new_h = np.append(new_h, end)

    elif GRID_MODE == "target_edges":
        edges = int(TARGET_NUM_EDGES)
        if edges < 1:
            raise ValueError("TARGET_NUM_EDGES must be >= 1.")
        points = edges + 1
        new_h = np.linspace(h_min, h_max, num=points)

    elif GRID_MODE == "scale_factor":
        extra = int(EXTRA_PER_EDGE)
        if extra < 0:
            raise ValueError("EXTRA_PER_EDGE must be >= 0.")
        # For each original edge [h_k, h_{k+1}], add `extra` internal points
        hs = []
        for k in range(len(base_heights) - 1):
            left, right = base_heights[k], base_heights[k+1]
            internal = np.linspace(left, right, num=extra + 2)[:-1]  # exclude right to avoid dup
            hs.append(internal)
        hs.append([base_heights[-1]])  # add the final endpoint
        new_h = np.concatenate(hs)

    else:
        raise ValueError("Unknown GRID_MODE. Use 'none', 'uniform_step', 'target_edges', or 'scale_factor'.")

    if INCLUDE_ORIGINAL:
        new_h = np.union1d(new_h, base_heights)

    # Final tidy up: sort & unique, clip to [h_min, h_max]
    new_h = np.asarray(sorted(set(new_h)))
    new_h = new_h[(new_h >= h_min - 1e-12) & (new_h <= h_max + 1e-12)]
    return new_h

new_heights = make_dense_heights(base["height_m"].to_numpy())

# =========================
# 3) INTERPOLATE SERIES ON NEW GRID
# =========================
x = base["height_m"].to_numpy()
y_dmg = base["damage_per_unit"].to_numpy()
y_cost = base["C_n"].to_numpy()

interp_damage = np.interp(new_heights, x, y_dmg)
interp_cost   = np.interp(new_heights, x, y_cost)

if ENFORCE_MONOTONE:
    interp_damage = np.maximum.accumulate(interp_damage)
    interp_cost   = np.maximum.accumulate(interp_cost)

dense = pd.DataFrame({
    "height_m": new_heights,
    "damage_per_unit": interp_damage,
    "C_n": interp_cost,
})

subset = dense.reset_index(drop=True).copy()
subset.attrs["N_UNITS"] = N_UNITS  # propagate n for downstream use

print(f"[Grid] Mode={GRID_MODE} | Points={subset.shape[0]} | Edges={subset.shape[0]-1}")
print(f"[Grid] N_UNITS (n) = {N_UNITS}")
display(subset.head(10).style.format({"height_m": f"{{:.{DEC_H}f}}",
                                      "damage_per_unit": f"{{:.{DEC_VAL}f}}",
                                      "C_n": f"{{:.{DEC_VAL}f}}" }))

# =========================
# 4) BUILD EDGES + MARGINS
# =========================
L = subset.shape[0]
if L < 2:
    raise ValueError("Refined grid has fewer than 2 points; increase density or check data.")

edges = pd.DataFrame({
    "k":   np.arange(L-1, dtype=int),
    "h_k":  subset["height_m"].iloc[:-1].to_numpy(),
    "h_k1": subset["height_m"].iloc[1:].to_numpy(),
    "C_k":  subset["C_n"].iloc[:-1].to_numpy(),
    "C_k1": subset["C_n"].iloc[1:].to_numpy(),
    "d_k":  subset["damage_per_unit"].iloc[:-1].to_numpy(),
    "d_k1": subset["damage_per_unit"].iloc[1:].to_numpy(),
})
edges["c_k"]     = edges["C_k1"] - edges["C_k"]
edges["m_plus"]  = edges["c_k"] - N_UNITS * edges["d_k1"]   # SAFE upward bound
edges["m_minus"] = N_UNITS * edges["d_k"] - edges["c_k"]    # DIAGNOSTIC ONLY (do not use for pruning)

print("\n[Margins] One-step margins on refined grid:")
display(edges[["k","h_k","h_k1","c_k","d_k","d_k1","m_plus","m_minus"]]
        .style.format({"h_k": f"{{:.{DEC_H}f}}", "h_k1": f"{{:.{DEC_H}f}}",
                        "c_k": f"{{:.{DEC_VAL}f}}", "d_k": f"{{:.{DEC_VAL}f}}",
                        "d_k1": f"{{:.{DEC_VAL}f}}", "m_plus": f"{{:.{DEC_VAL}f}}",
                        "m_minus": f"{{:.{DEC_VAL}f}}" }))

# =========================
# 5) PRUNING ROUTINES (UPWARD-ONLY, distribution-free)
# =========================
def local_prune(active_idx, edges_df, verbose=True, tol=1e-12):
    """Distribution-free safe local pruning using only m_plus."""
    to_remove = set()
    for _, row in edges_df.iterrows():
        k = int(row["k"])
        i, j = k, k+1
        if i in active_idx and j in active_idx:
            if row["m_plus"] > tol:
                to_remove.add(j)
                if verbose:
                    print(f"Local prune: remove index {j} (h={subset.loc[j,'height_m']:.{DEC_H}f}) "
                          f"since m_plus[{k}]={row['m_plus']:.{DEC_VAL}f}>0 ⇒ μ(h_j)>μ(h_i)")
            # Do NOT prune with m_minus (unsafe distribution-free)
    new_active = [t for t in active_idx if t not in to_remove]
    new_active.sort()
    changed = (set(new_active) != set(active_idx))
    return new_active, changed

def chain_prune(active_idx, edges_df, verbose=True, tol=1e-12):
    """Distribution-free safe chain pruning using cumulative sums of m_plus only."""
    to_remove = set()
    act = sorted(active_idx)
    m_plus = edges_df.set_index("k")["m_plus"]

    # prefix sums of m_plus on the full refined grid
    prefix_plus = np.zeros(L, dtype=float)
    for r in range(L-1):
        prefix_plus[r+1] = prefix_plus[r] + m_plus.get(r, 0.0)

    def sum_m_plus(j, k):
        # sum_{r=j}^{k-1} m_plus[r]
        return float(prefix_plus[k] - prefix_plus[j])

    for a in range(len(act)):
        for b in range(len(act)):
            if a == b:
                continue
            i, j = act[a], act[b]
            if i < j:
                s = sum_m_plus(i, j)
                if s > tol:
                    to_remove.add(j)
                    if verbose:
                        print(f"Chain prune: remove index {j} (h={subset.loc[j,'height_m']:.{DEC_H}f}) "
                              f"since Σ m_plus[{i}:{j-1}] = {s:.{DEC_VAL}f} > 0 ⇒ μ(h_j)>μ(h_i)")
            # No downward branch (unsafe distribution-free)
    new_active = [t for t in act if t not in to_remove]
    new_active.sort()
    changed = (set(new_active) != set(active_idx))
    return new_active, changed

def prune_all(edges_df, verbose=True, tol=1e-12):
    active = list(range(L))
    if verbose:
        print(f"\n[Prune] Initial active indices: {active}")
    changed = True
    iter_no = 0
    while changed:
        iter_no += 1
        if verbose:
            print(f"\n--- Iteration {iter_no}: local pruning ---")
        active, ch1 = local_prune(active, edges_df, verbose=verbose, tol=tol)
        if verbose:
            print(f"Active after local: {active}")

        if verbose:
            print(f"--- Iteration {iter_no}: chain pruning ---")
        active, ch2 = chain_prune(active, edges_df, verbose=verbose, tol=tol)
        if verbose:
            print(f"Active after chain: {active}")

        changed = ch1 or ch2

    survivors = subset.loc[active, ["height_m","damage_per_unit","C_n"]].copy()
    survivors["index"] = active
    survivors = survivors[["index","height_m","damage_per_unit","C_n"]].reset_index(drop=True)
    return active, survivors

# =========================
# 6) RUN PRUNING + Δ_min LOWER BOUND (UPWARD-ONLY)
# =========================
active_idx, survivors_df = prune_all(edges, verbose=True)

print("\n=== Surviving heights (after pruning) ===")
display(survivors_df.style.format({"height_m": f"{{:.{DEC_H}f}}",
                                   "damage_per_unit": f"{{:.{DEC_VAL}f}}",
                                   "C_n": f"{{:.{DEC_VAL}f}}" }))

# SAFE global Δ_min lower bound:
#   underline{Δ} = min_{j<k} (Σ_{r=j}^{k-1} m_plus[r])_+
m_plus_vals = edges["m_plus"].to_numpy()
prefix = np.zeros(L, dtype=float)
for r in range(L-1):
    prefix[r+1] = prefix[r] + m_plus_vals[r]

delta_vals = []
for j in range(L-1):
    # vectorized over k>j
    s = prefix[j+1:] - prefix[j]
    positive_s = s[s > 0]
    if positive_s.size:
        delta_vals.append(np.min(positive_s))

if len(delta_vals) == 0:
    delta_lb = np.nan
else:
    delta_lb = float(np.min(delta_vals))

print("\nΔ_min lower bound (distribution-free, upward-only):",
      "NaN (no positive cumulative margins)" if np.isnan(delta_lb) else f"{delta_lb:.{DEC_VAL}f}")


[Grid] Mode=target_edges | Points=81 | Edges=80
[Grid] N_UNITS (n) = 1


,height_m,damage_per_unit,C_n
0,0.000,0.000,0.000
1,0.075,0.481,1.923
2,0.150,0.962,3.846
3,0.225,1.444,5.769
4,0.300,1.925,7.693
5,0.375,2.406,9.616
6,0.450,2.887,11.539
7,0.525,3.541,14.052
8,0.600,4.539,17.746
9,0.675,5.538,21.439



[Margins] One-step margins on refined grid:


,k,h_k,h_k1,c_k,d_k,d_k1,m_plus,m_minus
0,0,0.000,0.075,1.923,0.000,0.481,1.442,-1.923
1,1,0.075,0.150,1.923,0.481,0.962,0.961,-1.442
2,2,0.150,0.225,1.923,0.962,1.444,0.480,-0.961
3,3,0.225,0.300,1.923,1.444,1.925,-0.002,-0.480
4,4,0.300,0.375,1.923,1.925,2.406,-0.483,0.002
5,5,0.375,0.450,1.923,2.406,2.887,-0.964,0.483
6,6,0.450,0.525,2.513,2.887,3.541,-1.028,0.374
7,7,0.525,0.600,3.694,3.541,4.539,-0.846,-0.153
8,8,0.600,0.675,3.694,4.539,5.538,-1.844,0.846
9,9,0.675,0.750,3.694,5.538,6.537,-2.843,1.844



[Prune] Initial active indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]

--- Iteration 1: local pruning ---
Local prune: remove index 1 (h=0.075) since m_plus[0]=1.442>0 ⇒ μ(h_j)>μ(h_i)
Local prune: remove index 2 (h=0.150) since m_plus[1]=0.961>0 ⇒ μ(h_j)>μ(h_i)
Local prune: remove index 3 (h=0.225) since m_plus[2]=0.480>0 ⇒ μ(h_j)>μ(h_i)
Active after local: [0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
--- Iteration 1: chain pruning ---
Chain prune: remove i

,index,height_m,damage_per_unit,C_n
0,0,0.000,0.000,0.000
1,8,0.600,4.539,17.746
2,9,0.675,5.538,21.439
3,10,0.750,6.537,25.133
4,11,0.825,7.535,28.827
5,12,0.900,8.534,32.520
6,13,0.975,9.532,36.214
7,14,1.050,11.087,41.023
8,15,1.125,12.921,46.389
9,16,1.200,14.755,51.755



Δ_min lower bound (distribution-free, upward-only): 0.406
